In [1]:
# Импорт модулей для работы с массивами
import numpy as np
import pandas as pd

# Импорт инструментов подготовки данных, кросвалидации и оценки
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import decomposition

# Импорт keras для MLP
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# Импорт данных
red = pd.read_csv('winequality-red.csv', sep =';')
white = pd.read_csv('winequality-white.csv', sep =';')

In [3]:
# преобразуем оценку качетсва вина (quality) 
#в три категории и посмотрим увеличится ли от этого точность работы алгоритмов

# 3 - плохое вино от 0 до 4
# 2 - среднее вино от 4 до 6 
# 1 - хорошее вино от 6 до 10

# Преобразуем диапазоны меток quality в 3 категории
rating_bins = [0, 4, 6, 10]
rating_labels = [3,2,1]

red['quality'] = pd.cut(red['quality'], bins=rating_bins, labels=rating_labels)

white['quality'] = pd.cut(white['quality'], bins=rating_bins, labels=rating_labels)

In [4]:
#отделим столбец с классами (оценкой качетсва quality) от признаков
feature = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide',
           'total sulfur dioxide','density','pH','sulphates','alcohol']

X_red = red[feature]
y_red = red['quality']

X_white = white[feature]
y_white = white['quality']

# Масштабирование признаков путём нормализации MinMax
X_red = MinMaxScaler().fit_transform(X_red) 
X_white = MinMaxScaler().fit_transform(X_white) 

In [5]:
# Разделим на обучающую и тестовую выборки
# Красное
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_red, y_red, test_size=0.3)

# Белое
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_white, y_white, test_size=0.3)

#### КРАСНОЕ

In [6]:
# Преобразуем метки в категории
Y_train_r = np_utils.to_categorical(y_train_r, 6)
Y_test_r = np_utils.to_categorical(y_test_r, 6)

# зададим последовательную модель нейронной сети
model_r = Sequential()

# Уровни сети (два полносвязных)
model_r.add(Dense(20, input_dim=11, activation='relu', kernel_initializer="normal"))
model_r.add(Dropout(0.2))
model_r.add(Dense(6, activation='softmax', kernel_initializer="normal"))

In [7]:
# class_weight = {3: 100,
#                 2: 1,
#                 1: 100}
# Компилим модель
model_r.compile(loss='categorical_crossentropy',
              optimizer='ADAM', metrics=['accuracy'])

print(model_r.summary())

# Обучение 
model_r.fit(X_train_r, Y_train_r, batch_size=400, epochs=300, validation_split=0.2, verbose=0)

# Оцениваем качество обучения сети на тестовых данных
scores = model_r.evaluate(X_test_r, Y_test_r, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                240       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 126       
Total params: 366
Trainable params: 366
Non-trainable params: 0
_________________________________________________________________
None
Точность работы на тестовых данных: 83.12%


#### БЕЛОЕ

In [8]:
# Преобразуем метки в категории
Y_train_w = np_utils.to_categorical(y_train_w, 7)
Y_test_w = np_utils.to_categorical(y_test_w, 7)

# зададим последовательную модель нейронной сети
model_w = Sequential()

# Уровни сети (два полносвязных)
model_w.add(Dense(20, input_dim=11, activation='relu', kernel_initializer="normal"))
model_w.add(Dropout(0.5))
model_w.add(Dense(7, activation='softmax', kernel_initializer="normal"))

In [9]:
# зададим веса классов
# class_weight = {3: 2.,
#                 2: 1.,
#                 1: 1.8}

# Компилим модель
model_w.compile(loss='categorical_crossentropy',
              optimizer='ADAM', metrics=['accuracy'])

print(model_w.summary())

# Обучение 
model_w.fit(X_train_w, Y_train_w, batch_size=20, epochs=20, validation_split=0.2, verbose=0)

# Оцениваем качество обучения сети на тестовых данных
scores = model_w.evaluate(X_test_w, Y_test_w, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 20)                240       
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 147       
Total params: 387
Trainable params: 387
Non-trainable params: 0
_________________________________________________________________
None
Точность работы на тестовых данных: 73.06%


##### MLP не обладает преимуществами в accuracy по сравнению с ансамблевыми методами, рассмотренными в ноутбуке 2. Быть может необходимы дополнительные тесты и более глубокое исследование.